In [18]:
import cv2
import cv2.aruco as aruco
import numpy as np

# --- kamera calibration (sementara dummy, ganti kalau sudah punya hasil kalibrasi) ---
camera_matrix = np.array([[800, 0, 320],
                          [0, 800, 240],
                          [0, 0, 1]], dtype=np.float32)
dist_coeffs = np.zeros((5,1))  # asumsi tanpa distorsi

# --- parameter marker ---
marker_length = 0.05  # meter (5 cm)

# --- buka webcam ---
cap = cv2.VideoCapture(0)

dictionary = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)
parameters = aruco.DetectorParameters()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # deteksi aruco
    corners, ids, rejected = aruco.detectMarkers(frame, dictionary, parameters=parameters)

    if ids is not None and len(ids) > 1:
        # estimasi pose tiap marker
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)

        # gambar marker di frame
        aruco.drawDetectedMarkers(frame, corners, ids)

        # simpan posisi (titik tengah marker di 3D, dalam meter)
        positions = {}
        centers_2d = {}
        for i, id in enumerate(ids.flatten()):
            positions[id] = tvecs[i][0]   # translasi (x,y,z)
            # ambil titik tengah marker di 2D (untuk gambar garis di layar)
            center_2d = np.mean(corners[i][0], axis=0).astype(int)
            centers_2d[id] = tuple(center_2d)

            # gambar axis di marker
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.03)

        # kalau ada ID 20 dan 23
        if 20 in positions and 23 in positions:
            p1 = positions[20]
            p2 = positions[23]

            # hitung beda X, Y, Z (dalam meter)
            diff = p2 - p1

            # ubah ke cm
            diff_cm = diff * 100
            dist_cm = np.linalg.norm(diff_cm)

            # tampilkan jarak di layar
            cv2.putText(frame, f"DX: {diff_cm[0]:.2f} cm", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            cv2.putText(frame, f"DY: {diff_cm[1]:.2f} cm", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            cv2.putText(frame, f"DZ: {diff_cm[2]:.2f} cm", (30,110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            cv2.putText(frame, f"Dist: {dist_cm:.2f} cm", (30,140), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

            # gambar garis antar titik tengah marker di 2D
            cv2.line(frame, centers_2d[20], centers_2d[23], (255,0,0), 2)

    cv2.imshow("Aruco Distance", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC keluar
        break

cap.release()
cv2.destroyAllWindows()


In [19]:
# Image Calibration: https://www.kaggle.com/code/danielwe14/stereocamera-calibration-with-opencv
# Lib AI: https://chatgpt.com/c/68dd7e2e-bae4-832f-879d-4c25ba40da8d